# Diagnostic Function

A power flow calculation on a pandapower network can fail to converge (or fail to run at all) for a vast variety of reasons, which often makes debugging difficult, annoying and time consuming. To help with that, the diagnostic function automatically checks pandapower networks for the most common issues leading to errors. It provides logging output and diagnoses with a controllable level of detail. 

## Example: faulty network

To demonstrate the usage of the diagnostic function we will use a very basic exemplary pandapower network with several flaws.

There will be no further explenation on how to create pandapower networks since there's a separate tutorial available in this regard.

In [1]:
import warnings
warnings.filterwarnings("ignore")

# imports the pandapower module
import pandapower as pp

# defines a function that creates an example network, which will be used a lot in this tutorial
# run this code first in order for the examples to work

def faulty_example_network():

    net = pp.create_empty_network()

    pp.create_bus(net, name = "110 kV bar", vn_kv = 110, type = 'b', in_service = 'True')
    pp.create_bus(net, name = "20 kV bar", vn_kv = 20, type = 'b')
    pp.create_bus(net, name = "bus 2", vn_kv = 30, type = 'b')
    pp.create_bus(net, name = "bus 3", vn_kv = 20, type = 'b')
    pp.create_bus(net, name = "bus 4", vn_kv = 20, type = 'b')
    pp.create_bus(net, name = "bus 5", vn_kv = -20, type = 'b')
    pp.create_bus(net, name = "bus 6", vn_kv = 20, type = 'b')
    
    pp.create_ext_grid(net, 0, vm_pu = 1)

    pp.create_line(net, name = "line 0", from_bus = 1, to_bus = 2, length_km = 0, std_type = "NAYY 4x150 SE")
    pp.create_line(net, name = "line 1", from_bus = 2, to_bus = 3, length_km = 1, std_type = "NAYY 4x150 SE")
    pp.create_line(net, name = "line 2", from_bus = 3, to_bus = 4, length_km = 1, std_type = "NAYY 4x150 SE")
    pp.create_line(net, name = "line 3", from_bus = 4, to_bus = 5, length_km = 1, std_type = "NAYY 4x150 SE")
    pp.create_line(net, name = "line 4", from_bus = 5, to_bus = 6, length_km = 1, std_type = "NAYY 4x150 SE")
    pp.create_line(net, name = "line 5", from_bus = 6, to_bus = 1, length_km = 1, std_type = "NAYY 4x150 SE")

    pp.create_transformer_from_parameters(net, hv_bus = 1, lv_bus = 0, i0_percent= 0.038, pfe_kw = 11.6,
                                          vkr_percent = 0.322, sn_mva = 40.0, vn_lv_kv = 22.0,
                                          vn_hv_kv = 110.0, vk_percent = 17.8)

    pp.create_load(net, 2, p_mw = -1, q_mvar = 0.200, name = "load 0")
    pp.create_load(net, 3, p_mw = 1, q_mvar = 0.200, name = "load 1")
    pp.create_load(net, 4, p_mw = 1, q_mvar = 0.200, name = "load 2")
    pp.create_load(net, 5, p_mw = 1, q_mvar = 0.200, name = "load 3")
    pp.create_load(net, 6, p_mw = 1, q_mvar = 0.200, name = "load 4")

    pp.create_switch(net, bus = 1, element = 0, et = 'l')
    pp.create_switch(net, bus = 2, element = 0, et = 'l')
    pp.create_switch(net, bus = 2, element = 1, et = 'l')
    pp.create_switch(net, bus = 3, element = 1, et = 'l')
    pp.create_switch(net, bus = 3, element = 2, et = 'l')
    pp.create_switch(net, bus = 4, element = 2, et = 'l')
    pp.create_switch(net, bus = 4, element = 3, et = 'l', closed = False)
    pp.create_switch(net, bus = 5, element = 3, et = 'l')
    pp.create_switch(net, bus = 5, element = 4, et = 'l', closed = False)
    pp.create_switch(net, bus = 6, element = 4, et = 'l', closed = False)
    pp.create_switch(net, bus = 6, element = 5, et = 'l')
    pp.create_switch(net, bus = 1, element = 5, et = 'l')
    
    return net

#### Now let's create the network and try to run a load flow calculation.

In [2]:
# creates the example network
faulty_net = faulty_example_network()
pp.runpp(faulty_net)

LoadflowNotConverged: Power Flow nr did not converge after 10 iterations!

As you can see there is at least one error in our network that prevents the load flow calculation from running properly.

#### Instead of analysing the code ourselves, we can use the diagnostic function to find the errors.

In [3]:
# diagnoses the faulty network
pp.diagnostic(faulty_net)

hp.pandapower.diagnostic_reports - WARNING: 

_____________ PANDAPOWER DIAGNOSTIC TOOL _____________ 

hp.pandapower.diagnostic_reports - INFO: Checking for missing bus indices...
hp.pandapower.diagnostic_reports - INFO: 
hp.pandapower.diagnostic_reports - INFO: PASSED: No missing bus indices found.
hp.pandapower.diagnostic_reports - WARNING: 
 --------

hp.pandapower.diagnostic_reports - WARNING: Checking for elements without a connection to an external grid...
hp.pandapower.diagnostic_reports - WARNING: 
hp.pandapower.diagnostic_reports - WARNING: Disconnected section found, consisting of the following elements:
hp.pandapower.diagnostic_reports - WARNING: buses: [5]
hp.pandapower.diagnostic_reports - WARNING: switches: [7, 8]
hp.pandapower.diagnostic_reports - WARNING: lines: [3]
hp.pandapower.diagnostic_reports - WARNING: loads: [3]
hp.pandapower.diagnostic_reports - WARNING: 
hp.pandapower.diagnostic_reports - WARNING: SUMMARY: 5 disconnected element(s) found.
hp.pandapower.diagnos

{'disconnected_elements': [{'buses': [5],
   'switches': [7, 8],
   'lines': [3],
   'loads': [3]}],
 'different_voltage_levels_connected': {'lines': [0, 1, 3, 4]},
 'impedance_values_close_to_zero': [{'line': [0]},
  {'loadflow_converges_with_switch_replacement': False}],
 'nominal_voltages_dont_match': {'trafo': {'hv_lv_swapped': [0]}},
 'invalid_values': {'bus': [(5, 'vn_kv', -20.0, '>0')],
  'line': [(0, 'length_km', 0.0, '>0')]},
 'overload': {'load': False, 'generation': False},
 'wrong_switch_configuration': False,
 'wrong_reference_system': {'loads': [0]}}

#### Customizing the report

For a more compact report, there a two options that can be passed as arguments to the diagnostic function:

- warnings_only = True : only positive check results (errors found) will be in the report

- detailed_report = False: only error summaries, no detailed error descriptions


The default setting is:

*diagnostic(net, warnings_only = False, detailed_report = True)*

In [4]:
# report with warnings_only
pp.diagnostic(faulty_net, warnings_only=True)

hp.pandapower.diagnostic_reports - WARNING: 

_____________ PANDAPOWER DIAGNOSTIC TOOL _____________ 

hp.pandapower.diagnostic_reports - WARNING: Checking for elements without a connection to an external grid...
hp.pandapower.diagnostic_reports - WARNING: 
hp.pandapower.diagnostic_reports - WARNING: Disconnected section found, consisting of the following elements:
hp.pandapower.diagnostic_reports - WARNING: buses: [5]
hp.pandapower.diagnostic_reports - WARNING: switches: [7, 8]
hp.pandapower.diagnostic_reports - WARNING: lines: [3]
hp.pandapower.diagnostic_reports - WARNING: loads: [3]
hp.pandapower.diagnostic_reports - WARNING: 
hp.pandapower.diagnostic_reports - WARNING: SUMMARY: 5 disconnected element(s) found.
hp.pandapower.diagnostic_reports - WARNING: 
 --------

hp.pandapower.diagnostic_reports - WARNING: Checking for connections of different voltage levels...
hp.pandapower.diagnostic_reports - WARNING: 
hp.pandapower.diagnostic_reports - WARNING: line 0 connects bus 1: 20 kV b

{'disconnected_elements': [{'buses': [5],
   'switches': [7, 8],
   'lines': [3],
   'loads': [3]}],
 'different_voltage_levels_connected': {'lines': [0, 1, 3, 4]},
 'impedance_values_close_to_zero': [{'line': [0]},
  {'loadflow_converges_with_switch_replacement': False}],
 'nominal_voltages_dont_match': {'trafo': {'hv_lv_swapped': [0]}},
 'invalid_values': {'bus': [(5, 'vn_kv', -20.0, '>0')],
  'line': [(0, 'length_km', 0.0, '>0')]},
 'overload': {'load': False, 'generation': False},
 'wrong_switch_configuration': False,
 'wrong_reference_system': {'loads': [0]}}

In [5]:
# report with summaries only
pp.diagnostic(faulty_net, report_style="compact")

hp.pandapower.diagnostic_reports - WARNING: 

_____________ PANDAPOWER DIAGNOSTIC TOOL _____________ 

hp.pandapower.diagnostic_reports - INFO: missing_bus_indices:
hp.pandapower.diagnostic_reports - INFO: 
hp.pandapower.diagnostic_reports - INFO: PASSED: No missing bus indices found.
hp.pandapower.diagnostic_reports - WARNING: 
 --------

hp.pandapower.diagnostic_reports - WARNING: disconnected_elements:
hp.pandapower.diagnostic_reports - WARNING: 
hp.pandapower.diagnostic_reports - WARNING: disonnected_section: {'buses': [5], 'switches': [7, 8], 'lines': [3], 'loads': [3]}
hp.pandapower.diagnostic_reports - WARNING: 
 --------

hp.pandapower.diagnostic_reports - WARNING: different_voltage_levels_connected:
hp.pandapower.diagnostic_reports - WARNING: 
hp.pandapower.diagnostic_reports - WARNING: lines:
hp.pandapower.diagnostic_reports - WARNING: line 0: buses [1, 2]
hp.pandapower.diagnostic_reports - WARNING: line 1: buses [2, 3]
hp.pandapower.diagnostic_reports - WARNING: line 3: buse

{'disconnected_elements': [{'buses': [5],
   'switches': [7, 8],
   'lines': [3],
   'loads': [3]}],
 'different_voltage_levels_connected': {'lines': [0, 1, 3, 4]},
 'impedance_values_close_to_zero': [{'line': [0]},
  {'loadflow_converges_with_switch_replacement': False}],
 'nominal_voltages_dont_match': {'trafo': {'hv_lv_swapped': [0]}},
 'invalid_values': {'bus': [(5, 'vn_kv', -20.0, '>0')],
  'line': [(0, 'length_km', 0.0, '>0')]},
 'overload': {'load': False, 'generation': False},
 'wrong_switch_configuration': False,
 'wrong_reference_system': {'loads': [0]}}

#### Let's have a look a the default report again

In [6]:
# diagnoses the faulty network
pp.diagnostic(faulty_net)

hp.pandapower.diagnostic_reports - WARNING: 

_____________ PANDAPOWER DIAGNOSTIC TOOL _____________ 

hp.pandapower.diagnostic_reports - INFO: Checking for missing bus indices...
hp.pandapower.diagnostic_reports - INFO: 
hp.pandapower.diagnostic_reports - INFO: PASSED: No missing bus indices found.
hp.pandapower.diagnostic_reports - WARNING: 
 --------

hp.pandapower.diagnostic_reports - WARNING: Checking for elements without a connection to an external grid...
hp.pandapower.diagnostic_reports - WARNING: 
hp.pandapower.diagnostic_reports - WARNING: Disconnected section found, consisting of the following elements:
hp.pandapower.diagnostic_reports - WARNING: buses: [5]
hp.pandapower.diagnostic_reports - WARNING: switches: [7, 8]
hp.pandapower.diagnostic_reports - WARNING: lines: [3]
hp.pandapower.diagnostic_reports - WARNING: loads: [3]
hp.pandapower.diagnostic_reports - WARNING: 
hp.pandapower.diagnostic_reports - WARNING: SUMMARY: 5 disconnected element(s) found.
hp.pandapower.diagnos

{'disconnected_elements': [{'buses': [5],
   'switches': [7, 8],
   'lines': [3],
   'loads': [3]}],
 'different_voltage_levels_connected': {'lines': [0, 1, 3, 4]},
 'impedance_values_close_to_zero': [{'line': [0]},
  {'loadflow_converges_with_switch_replacement': False}],
 'nominal_voltages_dont_match': {'trafo': {'hv_lv_swapped': [0]}},
 'invalid_values': {'bus': [(5, 'vn_kv', -20.0, '>0')],
  'line': [(0, 'length_km', 0.0, '>0')]},
 'overload': {'load': False, 'generation': False},
 'wrong_switch_configuration': False,
 'wrong_reference_system': {'loads': [0]}}

#### Disconnected elements

Bus 5, line 3, load 3 as well as switches 7 and 8 are not connected to an external grid. To fix that, we close switches 8 and 9.

In [7]:
faulty_net.switch.closed.loc[8, 9] = True

#### Inconsistent voltages

Lines 0, 1, 3 and 4 connect different voltage levels. Apparently bus 2 (30 kV) and bus 5 (-20 kV) have wrong voltage levels values. We change that to 20 kV.

In [8]:
faulty_net.bus.vn_kv.loc[2, 5] = 20

#### Lines with impedance zero

Line 0 is 0 km long (and therefore its impedance is 0 ohm). We change the length to 1 km.

In [9]:
faulty_net.line.length_km.at[0] = 1

#### Deviating nominal voltages

We fix that by swapping the hv and lv connectors of trafo 0 back.

In [10]:
faulty_net.trafo.hv_bus.at[0] = 0
faulty_net.trafo.lv_bus.at[0] = 1

#### Wrong reference system

Load 0 has a p_mw value of -1. Since pandapower uses the load reference system for loads, the value should either be positive or an sgen should be used. We assume the former and change the value to 1.

In [11]:
faulty_net.load.p_mw.at[0] = 1

In [12]:
# new diagnosis to check, whether we fixed all errors
pp.diagnostic(faulty_net)

hp.pandapower.diagnostic_reports - WARNING: 

_____________ PANDAPOWER DIAGNOSTIC TOOL _____________ 

hp.pandapower.diagnostic_reports - INFO: Checking for missing bus indices...
hp.pandapower.diagnostic_reports - INFO: 
hp.pandapower.diagnostic_reports - INFO: PASSED: No missing bus indices found.
hp.pandapower.diagnostic_reports - WARNING: 
 --------

hp.pandapower.diagnostic_reports - INFO: PASSED: No problematic switches found
hp.pandapower.diagnostic_reports - WARNING: 
 --------

hp.pandapower.diagnostic_reports - INFO: PASSED: No connection of different voltage levels found
hp.pandapower.diagnostic_reports - WARNING: 
 --------

hp.pandapower.diagnostic_reports - INFO: PASSED: No elements with impedance values close to zero found...
hp.pandapower.diagnostic_reports - WARNING: 
 --------

hp.pandapower.diagnostic_reports - INFO: PASSED: No components with deviating nominal voltages found
hp.pandapower.diagnostic_reports - WARNING: 
 --------

hp.pandapower.diagnostic_reports - I

{}

Apparently, all errors have been fixed. So we can try again to run a loadflow calculation.

In [13]:
pp.runpp(faulty_net)
faulty_net

This pandapower network includes the following parameter tables:
   - bus (7 elements)
   - load (5 elements)
   - switch (12 elements)
   - ext_grid (1 element)
   - line (6 elements)
   - trafo (1 element)
 and the following results tables:
   - res_bus (7 elements)
   - res_line (6 elements)
   - res_trafo (1 element)
   - res_ext_grid (1 element)
   - res_load (5 elements)
   - res_switch (12 elements)

Now the loadflow calculation runs without problems.

### Detecting overloads and wrong switch configurations

Other errors the diagnostic function can help to identify, are networks with too much load or wrong switch configurations. The function automatically checks, if the loadflow converges with all loads and generation scaled down to 0.1% or with all switches closed.

#### Example

In [14]:
# imports the pandapower module
import pandapower as pp

# defines a function that creates an example network
# run this code first in order for the examples to work

def overload_example_network():

    net = pp.create_empty_network()

    pp.create_bus(net, name = "110 kV bar", vn_kv = 110, type = 'b')
    pp.create_bus(net, name = "20 kV bar", vn_kv = 20, type = 'b')
    pp.create_bus(net, name = "bus 2", vn_kv = 20, type = 'b')
    pp.create_bus(net, name = "bus 3", vn_kv = 20, type = 'b')
    pp.create_bus(net, name = "bus 4", vn_kv = 20, type = 'b')
    pp.create_bus(net, name = "bus 5", vn_kv = 20, type = 'b')
    pp.create_bus(net, name = "bus 6", vn_kv = 20, type = 'b')
    
    pp.create_ext_grid(net, 0, vm_pu = 1)

    pp.create_line(net, name = "line 0", from_bus = 1, to_bus = 2, length_km = 1, std_type = "NAYY 4x150 SE")
    pp.create_line(net, name = "line 1", from_bus = 2, to_bus = 3, length_km = 1, std_type = "NAYY 4x150 SE")
    pp.create_line(net, name = "line 2", from_bus = 3, to_bus = 4, length_km = 1, std_type = "NAYY 4x150 SE")
    pp.create_line(net, name = "line 3", from_bus = 4, to_bus = 5, length_km = 1, std_type = "NAYY 4x150 SE")
    pp.create_line(net, name = "line 4", from_bus = 5, to_bus = 6, length_km = 1, std_type = "NAYY 4x150 SE")
    pp.create_line(net, name = "line 5", from_bus = 6, to_bus = 1, length_km = 1, std_type = "NAYY 4x150 SE")

    pp.create_transformer_from_parameters(net, hv_bus = 0, lv_bus = 1, i0_percent= 0.038, pfe_kw = 11.6,
        vkr_percent = 0.322, sn_mva = 40.0, vn_lv_kv = 22.0,
        vn_hv_kv = 110.0, vk_percent = 17.8)

    pp.create_load(net, 2, p_mw = 100, q_mvar = 0.2, name = "load 0")
    pp.create_load(net, 3, p_mw = 100, q_mvar = 0.2, name = "load 1")
    pp.create_load(net, 4, p_mw = 100, q_mvar = 0.2, name = "load 2")
    pp.create_load(net, 5, p_mw = 100, q_mvar = 0.2, name = "load 3")
    pp.create_load(net, 6, p_mw = 100, q_mvar = 0.2, name = "load 4")

    pp.create_switch(net, bus = 1, element = 0, et = 'l')
    pp.create_switch(net, bus = 2, element = 0, et = 'l')
    pp.create_switch(net, bus = 2, element = 1, et = 'l')
    pp.create_switch(net, bus = 3, element = 1, et = 'l')
    pp.create_switch(net, bus = 3, element = 2, et = 'l')
    pp.create_switch(net, bus = 4, element = 2, et = 'l')
    pp.create_switch(net, bus = 4, element = 3, et = 'l', closed = 0)
    pp.create_switch(net, bus = 5, element = 3, et = 'l')
    pp.create_switch(net, bus = 5, element = 4, et = 'l')
    pp.create_switch(net, bus = 6, element = 4, et = 'l')
    pp.create_switch(net, bus = 6, element = 5, et = 'l')
    pp.create_switch(net, bus = 1, element = 5, et = 'l')
    
    return net

In [15]:
overload_net = overload_example_network()
pp.runpp(overload_net)

LoadflowNotConverged: Power Flow nr did not converge after 10 iterations!

In [16]:
pp.diagnostic(overload_net, warnings_only = True)

hp.pandapower.diagnostic_reports - WARNING: 

_____________ PANDAPOWER DIAGNOSTIC TOOL _____________ 

hp.pandapower.diagnostic_reports - WARNING: Checking for overload...
hp.pandapower.diagnostic_reports - WARNING: 
hp.pandapower.diagnostic_reports - WARNING: overload found: Power flow converges with load scaled down to 0.1 percent.
hp.pandapower.diagnostic_reports - WARNING: 
 --------

hp.pandapower.diagnostic_reports - WARNING: Checking switch configuration...
hp.pandapower.diagnostic_reports - WARNING: 
hp.pandapower.diagnostic_reports - WARNING: Power flow still does not converge with all switches closed.
hp.pandapower.diagnostic_reports - WARNING: 
 --------

hp.pandapower.diagnostic_reports - WARNING: _____________ END OF PANDAPOWER DIAGNOSTIC _____________ 


{'overload': {'load': True, 'generation': False},
 'wrong_switch_configuration': False}

As you can see, with loads scaled down, the loadflow converges, which helps to isolate the problem.